# LSTMを用いた文書分類

# 1.LSTMの概要

## 1-1.背景
時系列データを取り扱えるモデルとしてRNNを取り上げました。<br>
しかしRNNでは理論上入力系列全てを取り扱っていますが、入力近くでは勾配消失が生じ、実際は数ステップ分しか、取り扱えていないと言われています。<br>
そこでより長い時系列を扱うモデルとして考案されたのが**LSTM(Long Short Term Memory)**となります。

## 1-2. 構造
RNNで登場した**隠れ層$h_t$**に加え、**メモリセル$c_t$**と**３種類のGate**という機構が追加されています。<br>
以下LSTMの構造の概略図になります。


<img src='img/LSTM概要図.png' width=600>

## 各役割を理解しよう
LSTMではRNNの時のような時系列ごとに受け渡される情報が**隠れ層**と**メモリセル**の二つです。

### - メモリセル $c_{t}$
次のタイムステップに次のユニットに渡されるベクトル<br>
RNNでは$h_{t}$がその役割だったが、LSTMでは$h_{t}$と$c_{t}$の二つが担う<br>
新たな入力を得たり、忘れたりを行います。<br>
メモリセルは今までの情報を**記憶**していると擬似的に解釈できます。


### 隠れ層 $h_{t}$
基本的にこれが各タイムステップごとの特徴量として扱われます。<br>
入力$x_{t}$があったときの出力は$h_{t}$　<br>
メモリセルをtanhで-1~1に変換された後にx0~1の変換を加えられます。

### - ForgetGate
ForgetGateは前のセル状態をどれくらい残すか（忘れさせるか）というゲート（門）になります。<br>
Gateの出力はシグモイド関数をかけることで、0~1に制限されています。<br>
ForgetGateは入力と一つ前の隠れ層によって決定されます。<br>
メモリセルに最初にかけられることで、前の情報をどれだけ通過させるか（１なら完全保持、０なら完全忘却）を制御します。

### - InputGate
InputGateはセル状態に新しい情報をどれだけ与えるかを制限するゲートになります。<br>
Gateの出力はシグモイド関数をかけることで、0~1に制限されています。<br>
InputGateは入力と一つ前の隠れ層によって決定されます。<br>
RNNの時と同様と入力$tanh(W_{hc}h_{t-1}+W_{xc}x_t)$をどれだけメモリセルに伝えるか（重要でない情報は覚えさせる必要はない）を制御します。

### - OutputGate
OutputGateはセル状態から隠れ層を作る段階で用いられます。<br>
Gateの出力はシグモイド関数をかけることで、0~1に制限されています。<br>
OutputGateは入力と一つ前の隠れ層によって決定されます。<br>
次の隠れ層を決定するため$tanh(c_t)$の出力を制御します。

## 1-3. 学習させる重み

LSTM内で学習を行う**重み**は全部で**8個**あります。
- $W_{xc}$　(入力xに掛ける重み)
- $W_{hc}$(前stepのHidden Stateに掛ける重み)
- Input Gate　$W_{xi}$(入力xに掛ける重み),$W_{hi}$(前stepのHidden Stateに掛ける重み)
- Forgeet Gate　$W_{xf}$(入力xに掛ける重み),$W_{hf}$(前stepのHidden Stateに掛ける重み)
- Output Gate　$W_{xo}$(入力xに掛ける重み),$W_{hf}$(前stepのHidden Stateに掛ける重み)

### Question1
上記8つの重みのshapeを答えよ。なお入力xの次元数をin_dimとし、隠れ層の次元数をhid_dim,セル状態の次元数をcell_dimとする。



### Answer
- $W_{xc}$ (in_dim , cell_dim)
- $W_{hc}$ (hid_dim , cell_dim)
- Input Gate　$W_{xi}$ (in_dim , cell_dim), $W_{hi}$ (hid_dim , cell_dim)
- Forgeet Gate　$W_{xf}$ (in_dim , cell_dim), $W_{hf}$ (hid_dim , cell_dim)
- Output Gate　$W_{xo}$ (in_dim , cell_dim), $W_{hf}$ (hid_dim , cell_dim)

# 2.LSTMを用いた文書分類(keras)

## 2-0.データの確認

今回はIMDBという映画のレビューサイトのデータセットになります。<br>
一つ一つのレビューに対し、1(Positive)と0(Negative)のラベル付けがされたデータが25000件用意されています。

有名なデータセットのため、すでにkerasに用意されています。
kerasでは前処理が行われており、各レビューは単語のインデックス（整数）のシーケンスとしてエンコードされています。単語はデータセットにおいての出現頻度によってインデックスされています。

In [229]:
from keras.preprocessing import sequence
from keras.models import Model
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.datasets import imdb
from keras.layers import Input

In [230]:
# HyperParams
max_features = 20000   #学習に用いる単語数
maxlen = 80  #学習に用いる最大長(これ以上は省略する)
batch_size = 32

## 2-1.データの読み込み

In [231]:
print('Loading data...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

Loading data...
25000 train sequences
25000 test sequences


In [232]:
x_train[0]

[1,
 14,
 22,
 16,
 43,
 530,
 973,
 1622,
 1385,
 65,
 458,
 4468,
 66,
 3941,
 4,
 173,
 36,
 256,
 5,
 25,
 100,
 43,
 838,
 112,
 50,
 670,
 2,
 9,
 35,
 480,
 284,
 5,
 150,
 4,
 172,
 112,
 167,
 2,
 336,
 385,
 39,
 4,
 172,
 4536,
 1111,
 17,
 546,
 38,
 13,
 447,
 4,
 192,
 50,
 16,
 6,
 147,
 2025,
 19,
 14,
 22,
 4,
 1920,
 4613,
 469,
 4,
 22,
 71,
 87,
 12,
 16,
 43,
 530,
 38,
 76,
 15,
 13,
 1247,
 4,
 22,
 17,
 515,
 17,
 12,
 16,
 626,
 18,
 19193,
 5,
 62,
 386,
 12,
 8,
 316,
 8,
 106,
 5,
 4,
 2223,
 5244,
 16,
 480,
 66,
 3785,
 33,
 4,
 130,
 12,
 16,
 38,
 619,
 5,
 25,
 124,
 51,
 36,
 135,
 48,
 25,
 1415,
 33,
 6,
 22,
 12,
 215,
 28,
 77,
 52,
 5,
 14,
 407,
 16,
 82,
 10311,
 8,
 4,
 107,
 117,
 5952,
 15,
 256,
 4,
 2,
 7,
 3766,
 5,
 723,
 36,
 71,
 43,
 530,
 476,
 26,
 400,
 317,
 46,
 7,
 4,
 12118,
 1029,
 13,
 104,
 88,
 4,
 381,
 15,
 297,
 98,
 32,
 2071,
 56,
 26,
 141,
 6,
 194,
 7486,
 18,
 4,
 226,
 22,
 21,
 134,
 476,
 26,
 480,
 5,
 144,
 30,

## 2-2.ZeroPadding
LSTMのモデルを構築する上で、系列長を統一する必要があります。<br>
そこで一定の長さに満たない、文章は後ろにZeroをpaddingすることで、入力とします。

また、計算コスト削減のために、一定長さ以上の文章は途中までの入力とします。

In [233]:
print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

Pad sequences (samples x time)
x_train shape: (25000, 80)
x_test shape: (25000, 80)


In [234]:
x_train[5]

array([    0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     1,   778,   128,    74,    12,   630,   163,    15,
           4,  1766,  7982,  1051,     2,    32,    85,   156,    45,
          40,   148,   139,   121,   664,   665,    10,    10,  1361,
         173,     4,   749,     2,    16,  3804,     8,     4,   226,
          65,    12,    43,   127,    24, 15344,    10,    10], dtype=int32)

## 2-3. Embedding
入力に単語などを用いる場合、単語をベクトルに変換する必要があります。<br>
ベクトルに変換する方法はTF-IDFやWord2Vecなど色々あります。<br>
今回はEmbedding(埋め込み)というやり方でベクトルに落とし込みます。

- 1.入力をtokenに分ける（英語の場合はスペース区切り、日本語の場合は形態素解析など）
- 2.それぞれのtokenと1対1で対応するベクトルを用意する（最初はrandomで初期化）
- 3.入力としてその単語の代わりにそのベクトルを用いる。（**このEmbeddingMatrixも学習させる対象**）

## 2-4.構造の確認
<img src='img/lstmtextclassification.png' width=500>
LSTMの最終時刻での隠れ層を特徴量とし、全結合層に入れることで文書分類をします。

## モデルの構築

In [235]:
hid_dim = 128  #隠れ層の次元数
embed_size = 128 #単語の埋め込み次元数
out_size=1
import os 
#os.environ['CUDA_VISIBLE_DEVICES']='3'

In [236]:
inputs =Input(shape=(maxlen,))
emb= Embedding(max_features, embed_size)(inputs)  #Embedding層
lstm = LSTM(hid_dim, dropout=0.2, recurrent_dropout=0.2, return_sequences=False)(emb)
outputs = Dense(out_size, activation='sigmoid')(lstm)


model = Model(inputs, outputs)

In [237]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 80)                0         
_________________________________________________________________
embedding_11 (Embedding)     (None, 80, 128)           2560000   
_________________________________________________________________
lstm_4 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 129       
Total params: 2,691,713
Trainable params: 2,691,713
Non-trainable params: 0
_________________________________________________________________


In [238]:
#compile
model.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy'])

#### 学習

In [239]:
model.fit(x_train,y_train,
          batch_size=batch_size,
          epochs=3,
          validation_data=[x_test,y_test])

Train on 25000 samples, validate on 25000 samples
Epoch 1/3
25000/25000 [==============================] - 188s 8ms/step - loss: 0.4574 - acc: 0.7828 - val_loss: 0.3868 - val_acc: 0.8315
Epoch 2/3
25000/25000 [==============================] - 182s 7ms/step - loss: 0.3091 - acc: 0.8734 - val_loss: 0.3823 - val_acc: 0.8317
Epoch 3/3
25000/25000 [==============================] - 181s 7ms/step - loss: 0.2216 - acc: 0.9148 - val_loss: 0.4181 - val_acc: 0.8304


#### IDから元の文章の復元

In [240]:
word_to_id = imdb.get_word_index()

In [241]:
word_to_id

{'fawn': 34701,
 'tsukino': 52006,
 'nunnery': 52007,
 'sonja': 16816,
 'vani': 63951,
 'woods': 1408,
 'spiders': 16115,
 'hanging': 2345,
 'woody': 2289,
 'trawling': 52008,
 "hold's": 52009,
 'comically': 11307,
 'localized': 40830,
 'disobeying': 30568,
 "'royale": 52010,
 "harpo's": 40831,
 'canet': 52011,
 'aileen': 19313,
 'acurately': 52012,
 "diplomat's": 52013,
 'rickman': 25242,
 'arranged': 6746,
 'rumbustious': 52014,
 'familiarness': 52015,
 "spider'": 52016,
 'hahahah': 68804,
 "wood'": 52017,
 'transvestism': 40833,
 "hangin'": 34702,
 'bringing': 2338,
 'seamier': 40834,
 'wooded': 34703,
 'bravora': 52018,
 'grueling': 16817,
 'wooden': 1636,
 'wednesday': 16818,
 "'prix": 52019,
 'altagracia': 34704,
 'circuitry': 52020,
 'crotch': 11585,
 'busybody': 57766,
 "tart'n'tangy": 52021,
 'burgade': 14129,
 'thrace': 52023,
 "tom's": 11038,
 'snuggles': 52025,
 'francesco': 29114,
 'complainers': 52027,
 'templarios': 52125,
 '272': 40835,
 '273': 52028,
 'zaniacs': 52130,

In [242]:
word_to_id = {k:(v+INDEX_FROM) for k,v in word_to_id.items()}

In [243]:
word_to_id

{'fawn': 34704,
 'tsukino': 52009,
 'nunnery': 52010,
 'sonja': 16819,
 'vani': 63954,
 'woods': 1411,
 'spiders': 16118,
 'hanging': 2348,
 'woody': 2292,
 'trawling': 52011,
 "hold's": 52012,
 'comically': 11310,
 'localized': 40833,
 'disobeying': 30571,
 "'royale": 52013,
 "harpo's": 40834,
 'canet': 52014,
 'aileen': 19316,
 'acurately': 52015,
 "diplomat's": 52016,
 'rickman': 25245,
 'arranged': 6749,
 'rumbustious': 52017,
 'familiarness': 52018,
 "spider'": 52019,
 'hahahah': 68807,
 "wood'": 52020,
 'transvestism': 40836,
 "hangin'": 34705,
 'bringing': 2341,
 'seamier': 40837,
 'wooded': 34706,
 'bravora': 52021,
 'grueling': 16820,
 'wooden': 1639,
 'wednesday': 16821,
 "'prix": 52022,
 'altagracia': 34707,
 'circuitry': 52023,
 'crotch': 11588,
 'busybody': 57769,
 "tart'n'tangy": 52024,
 'burgade': 14132,
 'thrace': 52026,
 "tom's": 11041,
 'snuggles': 52028,
 'francesco': 29117,
 'complainers': 52030,
 'templarios': 52128,
 '272': 40838,
 '273': 52031,
 'zaniacs': 52133,

In [244]:
INDEX_FROM=3   

word_to_id = imdb.get_word_index()
word_to_id = {k:(v+INDEX_FROM) for k,v in word_to_id.items()}
word_to_id["<PAD>"] = 0
word_to_id["<START>"] = 1
word_to_id["<UNK>"] = 2

id_to_word = {value:key for key,value in word_to_id.items()}

In [245]:
id_to_word

{34704: 'fawn',
 52009: 'tsukino',
 52010: 'nunnery',
 16819: 'sonja',
 63954: 'vani',
 1411: 'woods',
 16118: 'spiders',
 2348: 'hanging',
 2292: 'woody',
 52011: 'trawling',
 52012: "hold's",
 11310: 'comically',
 40833: 'localized',
 30571: 'disobeying',
 52013: "'royale",
 40834: "harpo's",
 52014: 'canet',
 19316: 'aileen',
 52015: 'acurately',
 52016: "diplomat's",
 25245: 'rickman',
 6749: 'arranged',
 52017: 'rumbustious',
 52018: 'familiarness',
 52019: "spider'",
 68807: 'hahahah',
 52020: "wood'",
 40836: 'transvestism',
 34705: "hangin'",
 2341: 'bringing',
 40837: 'seamier',
 34706: 'wooded',
 52021: 'bravora',
 16820: 'grueling',
 1639: 'wooden',
 16821: 'wednesday',
 52022: "'prix",
 34707: 'altagracia',
 52023: 'circuitry',
 11588: 'crotch',
 57769: 'busybody',
 52024: "tart'n'tangy",
 14132: 'burgade',
 52026: 'thrace',
 11041: "tom's",
 52028: 'snuggles',
 29117: 'francesco',
 52030: 'complainers',
 52128: 'templarios',
 40838: '272',
 52031: '273',
 52133: 'zaniacs',

#### 結果

In [254]:
num=0
' '.join(id_to_word[id] for id in x_test[num])

"<PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <START> please give this one a miss br br kristy swanson and the rest of the cast rendered terrible performances the show is flat flat flat br br i don't know how michael madison could have allowed this one on his plate he almost seemed to know this wasn't going to work out and his performance was quite lacklustre so all you madison fans give this a miss"

In [257]:
model.predict(x_test[num:num+1]) #推論結果negative

array([[ 0.22484739]], dtype=float32)

In [258]:
y_test[num] #正解データnegative

array([ 0.], dtype=float32)

# 3.LSTMフルスクラッチ

In [247]:
import numpy as np
import tensorflow as tf
from sklearn.utils import shuffle

rng = np.random.RandomState(42)

In [248]:
y_train=np.array(y_train.reshape(-1,1), dtype = 'float32')
y_test=np.array(y_test.reshape(-1,1), dtype = 'float32')

###  各層クラスの実装

In [249]:
class Embedding:
    def __init__(self, vocab_size, emb_dim, scale=0.08):
        #self.V = tf.Variable(rng.randn(vocab_size, emb_dim).astype('float32') * scale, name='V')
        self.V = tf.Variable(tf.random_uniform([vocab_size, emb_dim], -1.0, 1.0),name="V")

    def f_prop(self, x):
        return tf.nn.embedding_lookup(self.V, x)
    

#### 3-2-2 Long short-term memory (LSTM)

In [250]:
class LSTM:
    def __init__(self, in_dim, hid_dim, m, h_0=None, c_0=None):
        self.in_dim = in_dim
        self.hid_dim = hid_dim

        # input gate
        self.W_xi = tf.Variable(tf.random_uniform([in_dim, hid_dim], minval=-0.08, maxval=0.08, dtype=tf.float32), name='W_xi')
        self.W_hi = tf.Variable(tf.random_uniform([hid_dim, hid_dim], minval=-0.08, maxval=0.08, dtype=tf.float32), name='W_hi')
        self.b_i  = tf.Variable(tf.random_uniform([hid_dim], minval=-0.08, maxval=0.08, dtype=tf.float32), name='b_i')
        
        # forget gate
        self.W_xf = tf.Variable(tf.random_uniform([in_dim, hid_dim], minval=-0.08, maxval=0.08, dtype=tf.float32), name='W_xf')
        self.W_hf = tf.Variable(tf.random_uniform([hid_dim, hid_dim], minval=-0.08, maxval=0.08, dtype=tf.float32), name='W_xf')
        self.b_f  = tf.Variable(tf.random_uniform([hid_dim], minval=-0.08, maxval=0.08, dtype=tf.float32), name='b_f')

        # output gate
        self.W_xo = tf.Variable(tf.random_uniform([in_dim, hid_dim], minval=-0.08, maxval=0.08, dtype=tf.float32), name='W_xo')
        self.W_ho = tf.Variable(tf.random_uniform([hid_dim, hid_dim], minval=-0.08, maxval=0.08, dtype=tf.float32), name='W_ho')
        self.b_o  = tf.Variable(tf.random_uniform([hid_dim], minval=-0.08, maxval=0.08, dtype=tf.float32), name='b_o')

        # cell state
        self.W_xc = tf.Variable(tf.random_uniform([in_dim, hid_dim], minval=-0.08, maxval=0.08, dtype=tf.float32), name='W_xc')
        self.W_hc = tf.Variable(tf.random_uniform([hid_dim, hid_dim], minval=-0.08, maxval=0.08, dtype=tf.float32), name='W_hc')
        self.b_c  = tf.Variable(tf.random_uniform([hid_dim], minval=-0.08, maxval=0.08, dtype=tf.float32), name='b_c')

        # initial state
        self.h_0 = h_0
        self.c_0 = c_0

        # mask
        self.m = m
    """（参考）RNNでの実装
    def f_prop(self, x):
        def fn(h_tm1, x_and_m):
            x = x_and_m[0]
            m = x_and_m[1]
            h_t = tf.nn.tanh(tf.matmul(h_tm1, self.W_re) + tf.matmul(x, self.W_in) + self.b_re)
            return m[:, None] * h_t + (1 - m[:, None]) * h_tm1 # Mask

        # shape: [batch_size, sentence_length, in_dim] -> shape: [sentence_length, batch_size, in_dim]
        _x = tf.transpose(x, perm=[1, 0, 2])
        # shape: [batch_size, sentence_length] -> shape: [sentence_length, batch_size]
        _m = tf.transpose(self.m)
        h_0 = tf.matmul(x[:, 0, :], tf.zeros([self.in_dim, self.hid_dim])) # Initial state
        
        h = tf.scan(fn=fn, elems=[_x, _m], initializer=h_0)
        
        return h[-1] # Take the last state
    """
    def f_prop(self, x):
        def fn(tm1, x_and_m):
            h_tm1 = tm1[0]      #hidden state
            c_tm1 = tm1[1]      #cell state
            x_t = x_and_m[0]   #入力
            m_t = x_and_m[1]  #マスク
            # input gate
            i_t = tf.nn.sigmoid(tf.matmul(x_t, self.W_xi) + tf.matmul(h_tm1, self.W_hi) + self.b_i)

            # forget gate
            f_t = tf.nn.sigmoid(tf.matmul(x_t, self.W_xf) + tf.matmul(h_tm1, self.W_hf) + self.b_f)

            # output gate
            o_t = tf.nn.sigmoid(tf.matmul(x_t, self.W_xo) + tf.matmul(h_tm1, self.W_ho) + self.b_o)

            # cell state  forget*前のcell + input* （入力×重み＋hidden*重み）
            c_t = f_t * c_tm1 + i_t * tf.nn.tanh(tf.matmul(x_t, self.W_xc) + tf.matmul(h_tm1, self.W_hc) + self.b_c)
            c_t = m_t[:, np.newaxis] * c_t + (1. - m_t[:, np.newaxis]) * c_tm1 # Mask

            # hidden state　output*tanh(今cell)
            h_t = o_t * tf.nn.tanh(c_t)
            h_t = m_t[:, np.newaxis] * h_t + (1. - m_t[:, np.newaxis]) * h_tm1 # Mask

            return [h_t, c_t]

        _x = tf.transpose(x, perm=[1, 0, 2])
        _m = tf.transpose(self.m)

        if self.h_0 == None:
            self.h_0 = tf.matmul(x[:, 0, :], tf.zeros([self.in_dim, self.hid_dim]))
        if self.c_0 == None:
            self.c_0 = tf.matmul(x[:, 0, :], tf.zeros([self.in_dim, self.hid_dim]))

        h, c = tf.scan(fn=fn, elems=[_x, _m], initializer=[self.h_0, self.c_0])
        return h[-1]

In [251]:
class Dense:
    def __init__(self, in_dim, out_dim, function=lambda x: x):
        # Xavier
        self.W = tf.Variable(rng.uniform(
                        low=-np.sqrt(6/(in_dim + out_dim)),
                        high=np.sqrt(6/(in_dim + out_dim)),
                        size=(in_dim, out_dim)
                    ).astype('float32'), name='W')
        self.b = tf.Variable(tf.zeros([out_dim], dtype=tf.float32), name='b')
        self.function = function

    def f_prop(self, x):
        return self.function(tf.matmul(x, self.W) + self.b)

### 3-3 計算グラフ構築 & パラメータの更新設定

In [304]:
vocab_size = len(word_to_id)
emb_dim = 128
hid_dim = 128
output_size=1

x = tf.placeholder(tf.int32, [None, None], name='x')
t = tf.placeholder(tf.float32, [None, 1], name="input_y")
m = tf.cast(tf.not_equal(x, 0), tf.float32)   #padding部分にマスク

def f_props(layers, x):
    for layer in layers:
        x = layer.f_prop(x)
    return x

layer = [
    Embedding(vocab_size, emb_dim),
    LSTM(emb_dim, hid_dim, m),
    Dense(hid_dim,output_size, tf.nn.sigmoid)
]

y = f_props(layer, x)
y_pred = tf.round(y)
cost = tf.reduce_mean(tf.reduce_sum(tf.nn.sigmoid_cross_entropy_with_logits(labels=t, logits=y)))
train = tf.train.AdamOptimizer().minimize(cost)


accuracy = tf.reduce_mean(tf.cast(tf.equal(y_pred,t),dtype=tf.float32))

In [306]:
sess.run(accuracy,feed_dict={x:x_test[:100],t:y_test[:100]})

0.5

In [307]:
n_epochs =3
batch_size = 32
n_batches = x_train.shape[0]//batch_size

sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)


print("学習開始")    
for epoch in range(n_epochs):
    X_train, Y_train = shuffle(x_train, y_train)
    acc = sess.run([accuracy],feed_dict={x:x_test, t: y_test})
    print(acc)
    for i in range(n_batches):
       
        start = i * batch_size
        end = start + batch_size
        
        x_batch = X_train[start:end]
        y_batch = Y_train[start:end]
        
        _, train_cost = sess.run([train, cost], feed_dict={x:x_batch, t: y_batch})
  
    acc = sess.run([accuracy],feed_dict={x:x_test, t: y_test})
    
    print(" epoch "+str(epoch+1))
    print(train_cost)
    print(acc)

学習開始
[0.50716001]
 epoch 1
22.1794
[0.50032002]
[0.50032002]


KeyboardInterrupt: 